In [1]:
import os
import math
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import gym

In [15]:
env = gym.make('CartPole-v1')

In [16]:
# env.seed(100)
env.action_space.seed(100)

[100]

In [17]:
env.observation_space

Box([-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38], [4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38], (4,), float32)

In [18]:
env.observation_space.low.astype(np.float16)

array([-4.8  ,   -inf, -0.419,   -inf], dtype=float16)

In [19]:
env.observation_space.high.astype(np.float16)

array([4.8  ,   inf, 0.419,   inf], dtype=float16)

In [20]:
# Initial state: cart position, cart velocity, pole angle, pole angular velocity
state = env.reset()
state

(array([-0.04974512, -0.00799374, -0.03035298,  0.03352895], dtype=float32),
 {})

In [21]:
# Action은 0(왼쪽으로 밀기), 1(오른쪽으로 밀기)

In [22]:
env.action_space

Discrete(2)

In [23]:
env.action_space.n

2

In [24]:
env.action_space.sample()

1

In [25]:
env.action_space.sample()

1

In [26]:
a = env.action_space.sample()
a

0

In [29]:
state, reward, done, truncated, info = env.step(a)
state, reward, done, info

(array([-0.05410106, -0.20191507, -0.02306097,  0.29981315], dtype=float32),
 1.0,
 False,
 {})

In [31]:
env.reset()
for e in range(1, 200):
    a = env.action_space.sample()
    state, reward, done, truncated, info = env.step(a)
    print(f'step={e:2d} | state={state} | action={a} | reward={reward}')
    if done and (e + 1) < 200:
        print('*** FAILED ***')
        break

step= 1 | state=[ 0.0175098  -0.24520719  0.04880392  0.29376006] | action=0 | reward=1.0
step= 2 | state=[ 0.01260565 -0.44098976  0.05467912  0.6014269 ] | action=0 | reward=1.0
step= 3 | state=[ 0.00378586 -0.63683224  0.06670766  0.91081923] | action=0 | reward=1.0
step= 4 | state=[-0.00895079 -0.8327904   0.08492404  1.2237002 ] | action=0 | reward=1.0
step= 5 | state=[-0.02560659 -0.6388587   0.10939805  0.9587887 ] | action=1 | reward=1.0
step= 6 | state=[-0.03838377 -0.44536403  0.12857382  0.70238066] | action=1 | reward=1.0
step= 7 | state=[-0.04729105 -0.25223637  0.14262144  0.45277703] | action=1 | reward=1.0
step= 8 | state=[-0.05233578 -0.05938885  0.15167698  0.20823146] | action=1 | reward=1.0
step= 9 | state=[-0.05352355 -0.2563175   0.1558416   0.5446553 ] | action=0 | reward=1.0
step=10 | state=[-0.0586499  -0.06368917  0.16673471  0.3048466 ] | action=1 | reward=1.0
step=11 | state=[-0.05992369  0.1287129   0.17283164  0.06903905] | action=1 | reward=1.0
step=12 | 

In [32]:
done

True

In [33]:
# Monte Carlo Agent

In [46]:
np.random.seed(100)
weights = np.random.random(4) * 2 - 1
weights

array([ 0.08680988, -0.44326123, -0.15096482,  0.68955226])

In [47]:
state = env.reset()
state[0]

array([-0.01492935,  0.00995991,  0.02342201,  0.00863748], dtype=float32)

In [48]:
s = np.dot(state[0], weights)
s

-0.0032907612043148616

In [49]:
if s < 0:
    a = 0
else:
    a = 1
    
a

0

In [50]:
def run_episode(env, weights):
    state = env.reset()
    state = state[0]
    treward = 0
    for _ in range(200):
        s = np.dot(state, weights)
        a = 0 if s < 0 else 1
        state, reward, done, truncated, info = env.step(a)
        treward += reward
        if done:
            break
    return treward

In [51]:
run_episode(env, weights)

58.0

In [53]:
def set_seeds(seed=100):
    random.seed(seed)
    np.random.seed(seed)
    # env.seed(seed)

In [57]:
set_seeds()
num_episodes = 1000

In [58]:
besttreward = 0
for e in range(1, num_episodes + 1):
    weights = np.random.rand(4) * 2 - 1    # random weights
    treward = run_episode(env, weights)    # Total reward for these weights
    if treward > besttreward:             # 상승했는지?
        besttreward = treward
        bestweights = weights
        if treward == 200:
            print(f'SUCESS | episode={e}')
            break
        print(f'UPDATE | episode={e}')

UPDATE | episode=1
UPDATE | episode=2
UPDATE | episode=10
SUCESS | episode=13


In [59]:
weights

array([-0.42820862,  0.70479018,  0.95001299,  0.76970659])

In [60]:
res = []
for _ in range(100):
    treward = run_episode(env, weights)
    res.append(treward)
res[:10]

[200.0, 200.0, 200.0, 200.0, 200.0, 200.0, 200.0, 200.0, 200.0, 200.0]

In [61]:
sum(res) / len(res)

200.0

In [62]:
# Neural Network Agent

In [63]:
import tensorflow as tf
from keras.layers import Dense, Dropout
from keras.models import Sequential
from keras.optimizers import Adam, RMSprop
from sklearn.metrics import accuracy_score

In [64]:
def set_seeds(seed=100):
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    # env.seed(seed)
    env.action_space.seed(100)

In [65]:
class NNAgent:
    def __init__(self):
        self.max = 0
        self.scores = list()
        self.memory = list()
        self.model = self._build_model()
        
    def _build_model(self):
        model = Sequential()
        model.add(Dense(24, input_dim=4,
                       activation='relu'))
        model.add(Dense(1, activation='sigmoid'))
        model.compile(loss='binary_crossentropy',
                     optimizer=RMSprop(lr=0.001))
        return model
    
    def act(self, state):
        if random.random() <= 0.5:
            return env.action_space.sample()
        action = np.where(self.model.predict(
            state, batch_size=None)[0, 0] > 0.5, 1, 0)
        return action
    
    def train_model(self, state, action):
        self.model.fit(state, np.array([action,]),
                      epochs=1, verbose=False)
        
    def learn(self, episodes):
        for e in range(1, episodes + 1):
            state = env.reset()
            state = state[0]
            for _ in range(201):
                state = np.reshape(state, [1, 4])
                action = self.act(state)
                next_state, reward, done, truncated, info = env.step(action)
                if done:
                    score = _ + 1
                    self.scores.append(score)
                    self.max = max(score, self.max)
                    print('episode: {:4d}/{} | score: {:3d} | max: {:3d}'
                            .format(e, episodes, score, self.max), end='\r')
                    break
                self.memory.append((state, action))
                self.train_model(state, action)
                state= next_state

In [66]:
set_seeds(100)
agent = NNAgent()

episodes = 500

C:\Users\hong9\anaconda3\envs\q_env\lib\site-packages\keras\optimizers\optimizer_v2\rmsprop.py:140: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [67]:
agent.learn(episodes)

1/1 [==============================] - 0s 60ms/step


1/1 [==============================] - 0s 50ms/step


1/1 [==============================] - 0s 44ms/step


1/1 [==============================] - 0s 46ms/step


1/1 [==============================] - 0s 33ms/step


...

1/1 [==============================] - 0s 41ms/step


1/1 [==============================] - 0s 41ms/step


1/1 [==============================] - 0s 37ms/step


1/1 [==============================] - 0s 39ms/step


KeyboardInterrupt: 

In [68]:
sum(agent.scores) / len(agent.scores)

45.63194444444444

In [69]:
f = np.array([m[0][0] for m in agent.memory])
f

array([[ 0.04584129,  0.01795867, -0.00601151,  0.02816377],
       [ 0.04620046,  0.21316631, -0.00544823, -0.26640978],
       [ 0.05046379,  0.01812254, -0.01077643,  0.02454977],
       ...,
       [ 0.12337235, -0.19747072, -0.14465876, -0.08062379],
       [ 0.11942293, -0.00060323, -0.14627123, -0.41522214],
       [ 0.11941087, -0.1933821 , -0.15457568, -0.17199232]],
      dtype=float32)

In [ ]:
l = np.array([m[1] for m in agent.memory])
l

In [ ]:
accuracy_score(np.where(agent.model.predict(f) > 0.5, 1, 0), l)

In [ ]:
# DQL Agent

In [70]:
from collections import deque
from keras.optimizers import Adam, RMSprop

In [100]:
class DQLAgent:
    def __init__(self, gamma=0.95, hu=24, opt=Adam,
                lr=0.001, finish=False):
        self.finish = finish
        self.epsilon = 1.0           # Initial exploration rate
        self.epsilon_min = 0.01      # Minimum exploration rate
        self.epsilon_decay = 0.995   # Decay rate for exploration rate
        self.gamma = gamma           # Discount factor for delayed reward
        self.batch_size = 32         # Batch size for replay
        self.max_treward = 0
        self.averages = list()
        self.memory = deque(maxlen=2000)  # deque collection for limited history
        self.osn = env.observation_space.shape[0]
        self.model = self._build_model(hu, opt, lr)
        
    def _build_model(self, hu, opt, lr):
        model = Sequential()
        model.add(Dense(hu, input_dim=self.osn,
                        activation='relu'))
        model.add(Dense(hu, activation='relu'))
        model.add(Dense(env.action_space.n, activation='linear'))
        model.compile(loss='mse', optimizer=opt(lr=lr))
        return model
    
    def act(self, state):
        if random.random() <= self.epsilon:
            return env.action_space.sample()
        action = self.model.predict(state)[0]
        return np.argmax(action)
    
    def replay(self):
        batch = random.sample(self.memory, self.batch_size) # Random selection of history batch for replay
        for state, action, reward, next_state, done in batch:
            if not done:
                reward += self.gamma * np.amax(self.model.predict(next_state)[0]) # Q value for state-action pair
            target = self.model.predict(state)
            target[0, action] = reward
            self.model.fit(state, target, epochs=1, verbose=False) # Update of the neural network for the new action-value pairs
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay  # Update of the exploration rate
            
    def learn(self, episodes):
        trewards = []
        for e in range(1, episodes + 1):
            state = env.reset()
            state = np.reshape(state, [1, self.osn])
            for _ in range(5000):
                action = self.act(state)
                next_state, reward, done, info = env.step(action)
                next_state = np.reshape(next_state, [1, self.osn])
                self.memory.append([state, action, reward, next_state, done])  # Storing the new data
                state = next_state
                if done:
                    treward = _ + 1
                    trewards.append(treward)
                    av = sum(trewards[-25:]) / 25
                    self.averages.append(av)
                    self.max_treward = max(self.max_treward, treward)
                    templ = 'episode: {:4d}/{} | treward: {:4d} | '
                    templ += 'av: {:6.1f} | max: {:4d}'
                    print(templ.format(e, episodes, treward, av, self.max_treward), end='\r')
                    break
            if av > 195 and self.finish:
                break
            if len(self.memory) > self.batch_size:
                self.replay()  # Replay to update the policy based on past experiences
    
    def test(self, episodes):
        trewards = []
        for e in range(1, episodes + 1):
            state = env.reset()
            for _ in range(5001):
                state = np.reshape(state, [1, self.osn])
                action = np.argmax(self.model.predict(state)[0])
                next_state, reward, done, info = env.step(action)
                state = next_state
                if done:
                    treward = _ + 1
                    trewards.append(treward)
                    print('episode: {:4d}/{} | treward: {:4d}'.format(e, episodes, treward), end='\r')
                    break
        return trewards


In [78]:
episodes = 1000
set_seeds(100)
agent= DQLAgent(finish=True)

In [79]:
agent.learn(episodes)

1/1 [==============================] - 0s 39ms/step


1/1 [==============================] - 0s 49ms/step


Exception ignored in: <function UniquePtr.__del__ at 0x00000284820FE700>
Traceback (most recent call last):
  File "C:\Users\hong9\anaconda3\envs\q_env\lib\site-packages\tensorflow\python\framework\c_api_util.py", line 71, in __del__
    obj = self._obj
AttributeError: _obj


1/1 [==============================] - 0s 29ms/step


1/1 [==============================] - 0s 29ms/step


...

1/1 [==============================] - 0s 48ms/step


1/1 [==============================] - 0s 53ms/step


1/1 [==============================] - 0s 57ms/step


1/1 [==============================] - 0s 54ms/step


KeyboardInterrupt: 

In [ ]:
plt.figure(figsize=(10, 6))
x = range(len(agent.averages))
y = np.polyval(np.polyfit(x, agent.averages, deg=3), x)
plt.plot(agent.averages, label='moving average')
plt.plot(x, y, 'r--', label='trend')
plt.xlabel('episodes')
plt.ylabel('total reward')
plt.legend();

In [ ]:
trewards = agent.test(100)

In [ ]:
sum(trewards) / len(trewards)

In [ ]:
# Simple Finance Gym

In [101]:
class observation_space:
    def __init__(self, n):
        self.shape = (n, )
        
class action_space:
    def __init__(self, n):
        self.n = n
    def seed(self, seed):
        pass
    def sample(self):
        return random.randint(0, self.n - 1)

In [102]:
class Finance:
    url = 'http://hilpisch.com/aiif_eikon_eod_data.csv'
    def __init__(self, symbol, features):
        self.symbol = symbol
        self.features = features
        self.observation_space = observation_space(4)
        self.osn = self.observation_space.shape[0]
        self.action_space = action_space(2)
        self.min_accuracy = 0.475
        self._get_data()
        self._prepare_data()
    
    def _get_data(self):
        self.raw = pd.read_csv(self.url, index_col=0, parse_dates=True).dropna()
        
    def _prepare_data(self):
        self.data = pd.DataFrame(self.raw[self.symbol])
        self.data['r'] = np.log(self.data / self.data.shift(1))
        self.data.dropna(inplace=True)
        self.data = (self.data - self.data.mean()) / self.data.std()
        self.data['d'] = np.where(self.data['r'] > 0, 1, 0)
        
    def _get_state(self):
        # Selects the data defining the state of the financial market.
        return self.data[self.features].iloc[self.bar - self.osn : self.bar].values
    
    def seed(self, seed=None):
        pass
    def reset(self):
        self.treward = 0
        self.accuracy = 0
        self.bar = self.osn
        state = self.data[self.features].iloc[self.bar - self.osn : self.bar]
        return state.values
    
    def step(self, action):
        correct = action == self.data['d'].iloc[self.bar]
        reward = 1 if correct else 0
        self.treward += reward
        self.bar += 1    # Moves the environment one step forward
        # Calculates the accuracy of successful actions (trades) given all steps (trades)
        self.accuracy = self.treward / (self.bar - self.osn)
        # If the agent reaches the end of the data set, success is achieved.
        if self.bar >= len(self.data):
            done = True
        # If the agent takes the right action, it can move on
        elif reward == 1:
            done = False
        # If, after some initial steps, the accuracy drops under the minimum level, the episode ends (failure).
        elif (self.accuracy < self.min_accuracy and self.bar > self.osn + 10):
            done = True
        # For the remaining cases, the agent can move on.
        else:
            done = False
        state = self._get_state()
        info = {}
        return state, reward, done, info

In [103]:
env = Finance('EUR=', 'EUR=')

In [104]:
env.reset()

array([1.81901285, 1.85786346, 1.77486442, 1.85786346])

In [105]:
a = env.action_space.sample()
a

0

In [106]:
env.step(a)

(array([1.85786346, 1.77486442, 1.85786346, 1.94704329]), 0, False, {})

In [107]:
set_seeds(100)
agent = DQLAgent(gamma=0.5, opt=RMSprop)

C:\Users\hong9\anaconda3\envs\q_env\lib\site-packages\keras\optimizers\optimizer_v2\rmsprop.py:140: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [108]:
episodes = 1000

In [109]:
agent.learn(episodes)

1/1 [==============================] - 0s 44ms/step


1/1 [==============================] - 0s 55ms/step


1/1 [==============================] - 0s 46ms/step


1/1 [==============================] - 0s 68ms/step


KeyboardInterrupt: 

In [ ]:
agent.test(3)

In [ ]:
plt.figure(figsize=(10, 6))
x = range(len(agent.averages))
y = np.polyval(np.polyfit(x, agent.averages, deg=3), x)
plt.plot(agent.averages, label='moving average')
plt.plot(x, y, 'r--', label='regression')
plt.xlabel('episodes')
plt.ylabel('total reward')
plt.legend();

In [ ]:
# Better FinanceGym

In [110]:
class Finance:
    url = 'http://hilpisch.com/aiif_eikon_eod_data.csv'
    def __init__(self, symbol, features, window, lags,
                leverage=1, min_performance=0.85,
                start=0, end=None, mu=None, std=None):
        self.symbol = symbol
        self.features = features   # The features to define the state
        self.n_features = len(features)
        self.window = window
        self.lags = lags
        self.leverage = leverage  # The number of lags to be used
        self.min_performance = min_performance # The minimum gross performance required
        self.start = start
        self.end = end
        self.mu = mu
        self.std = std
        self.observation_space = observation_space(self.lags)
        self.action_space = action_space(2)
        self._get_data()
        self._prepare_data()
    def _get_data(self):
        self.raw = pd.read_csv(self.url, index_col=0, parse_dates=True).dropna()
    def _prepare_data(self):
        self.data = pd.DataFrame(self.raw[self.symbol])
        self.data = self.data.iloc[self.start:]
        self.data['r'] = np.log(self.data / self.data.shift(1))
        self.data.dropna(inplace=True)
        # Additional financial features
        self.data['s'] = self.data[self.symbol].rolling(self.window).mean()
        self.data['m'] = self.data['r'].rolling(self.window).mean()
        self.data['v'] = self.data['r'].rolling(self.window).std()
        self.data.dropna(inplace=True)
        # Gaussian normalization of the data
        if self.mu is None:
            self.mu = self.data.mean()
            self.std = self.data.std()
        self.data_ = (self.data - self.mu) / self.std
        self.data_['d'] = np.where(self.data['r'] > 0, 1, 0)
        self.data_['d'] = self.data_['d'].astype(int)
        if self.end is not None:
            self.data = self.data.iloc[:self.end - self.start]
            self.data_ = self.data_.iloc[:self.end - self.start]
    def _get_state(self):
        return self.data_[self.features].iloc[self.bar - self.lags:self.bar]
    
    def seed(self, seed):
        random.seed(seed)
        np.random.seed(seed)
    def reset(self):
        self.treward = 0
        self.accuracy = 0
        self.performance = 1
        self.bar = self.lags
        state = self.data_[self.features].iloc[self.bar - self.lags:self.bar]
        return state.values
    def step(self, action):
        correct = action == self.data_['d'].iloc[self.bar]
        ret = self.data['r'].iloc[self.bar] * self.leverage   # The leveraged return for the step
        reward_1 = 1 if correct else 0
        reward_2 = abs(ret) if correct else -abs(ret)   # The return-based reward for the step
        factor = 1 if correct else -1
        self.treward += reward_1
        self.bar +=1
        self.accuracy = self.treward / (self.bar - self.lags)
        self.performance *= math.exp(reward_2)    # The gross performance after the step
        if self.bar >= len(self.data):
            done = True
        elif reward_1 == 1:
            done = False
        elif (self.performance < self.min_performance and self.bar > self.lags + 5):
            done = True
        else:
            done = False
        state = self._get_state()
        info = {}
        return state.values, reward_1 + reward_2 * 5, done, info
        

In [111]:
env = Finance('EUR=', ['EUR=', 'r'], 10, 5)

In [112]:
a = env.action_space.sample()
a

1

In [113]:
env.reset()

array([[ 1.77206055, -1.02135357],
       [ 1.59731584, -2.44318439],
       [ 1.58755852, -0.12078195],
       [ 1.62924888,  0.60830363],
       [ 1.64078026,  0.18066537]])

In [114]:
env.step(a)

(array([[ 1.59731584, -2.44318439],
        [ 1.58755852, -0.12078195],
        [ 1.62924888,  0.60830363],
        [ 1.64078026,  0.18066537],
        [ 1.57247903, -0.95019257]]),
 -0.027282780374079853,
 False,
 {})

In [142]:
class FQLAgent:
    def __init__(self, hidden_units, learning_rate, learn_env, valid_env):
        self.learn_env = learn_env
        self.valid_env = valid_env
        self.epsilon = 1.0             # Initial exploration rate
        self.epsilon_min = 0.1         # Minimum exploration rate
        self.epsilon_decay = 0.98      # Decay rate for exploration rate
        self.learning_rate = learning_rate
        self.gamma = 0.95              # Discount factor for delayed reward
        self.batch_size = 128          # Batch size for replay
        self.max_treward = 0
        self.trewards = list()
        self.averages = list()
        self.performances = list()
        self.aperformances = list()
        self.vperformances = list()
        self.memory = deque(maxlen=2000) # deque collection for limited history
        self.model = self._build_model(hidden_units, learning_rate)
        
    def _build_model(self, hu, lr):
        model = Sequential()
        model.add(Dense(hu, input_shape=(
                self.learn_env.lags, self.learn_env.n_features),
                activation='relu'))
        model.add(Dropout(0.3, seed=100))
        model.add(Dense(hu, activation='relu'))
        model.add(Dropout(0.3, seed=100))
        model.add(Dense(2, activation='linear'))
        model.compile(loss='mse', optimizer=RMSprop(lr=lr))
        return model
    
    def act(self, state):
        if random.random() <= self.epsilon:
            return self.learn_env.action_space.sample()
        action = self.model.predict(state)[0, 0]
        return np.argmax(action)

    
    def replay(self):
        batch = random.sample(self.memory, self.batch_size)
        for state, action, reward, next_state, done in batch:
            if not done:
                reward += self.gamma * np.amax(self.model.predict(next_state)[0, 0])
            target = self.model.predict(state)
            target[0, 0, action] = reward
            self.model.fit(state, target, epochs=1, verbose=False)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay
            
    def learn(self, episodes):
        for e in range(1, episodes + 1):
            state = self.learn_env.reset()
            state = np.reshape(state, [1, self.learn_env.lags, self.learn_env.n_features])
            for _ in range(10000):
                action = self.act(state)
                next_state, reward, done, info = self.learn_env.step(action)
                next_state = np.reshape(next_state, [1, self.learn_env.lags, self.learn_env.n_features])
                self.memory.append([state, action, reward, next_state, done])
                state = next_state
                if done:
                    treward = _ + 1
                    self.trewards.append(treward)
                    av = sum(self.trewards[-25:]) / 25
                    perf = self.learn_env.performance
                    self.averages.append(av)
                    self.performances.append(perf)
                    self.aperformances.append(sum(self.performances[-25:]) / 25)
                    self.max_treward = max(self.max_treward, treward)
                    templ = 'episode: {:2d}/{} | treward: {:4d} | '
                    templ += 'perf: {:5.3f} | av: {:5.1f} | max: {:4d}'
                    print(templ.format(e, episodes, treward, perf,
                    av, self.max_treward), end='\r')
                    break
            self.validate(e, episodes)
            if len(self.memory) > self.batch_size:
                self.replay()
    
    def validate(self, e, episodes):
        state = self.valid_env.reset()
        state = np.reshape(state, [1, self.valid_env.lags, self.valid_env.n_features])
        for _ in range(10000):
            action = np.argmax(self.model.predict(state)[0, 0])
            next_state, reward, done, info = self.valid_env.step(action)
            state = np.reshape(next_state, [1, self.valid_env.lags, self.valid_env.n_features])
            if done:
                treward = _ + 1
                perf = self.valid_env.performance
                self.vperformances.append(perf)
                if e % 20 == 0:
                    templ = 71 * '='
                    templ += '\nepisode: {:2d}/{} | VALIDATION | '
                    templ += 'treward: {:4d} | perf: {:5.3f} | '
                    templ += 'eps: {:.2f}\n'
                    templ += 71 * '='
                    print(templ.format(e, episodes, treward, perf, self.epsilon))
                break

    
    def test(self, episodes):
        trewards = []
        for e in range(1, episodes + 1):
            state = env.reset()
            for _ in range(5001):
                state = np.reshape(state, [1, self.osn])
                action = np.argmax(self.model.predict(state)[0])
                next_state, reward, done, info = env.step(action)
                state = next_state
                if done:
                    treward = _ + 1
                    trewards.append(treward)
                    print('episode: {:4d}/{} | treward: {:4d}'.format(e, episodes, treward), end='\r')
                    break
        return trewards


In [143]:
symbol = 'EUR='
features = [symbol, 'r', 's', 'm', 'v']
a = 0
b = 2000
c = 500

In [144]:
learn_env = Finance(symbol, features, window=10, lags=6,
                    leverage=1, min_performance=0.85,
                    start=a, end=a + b, mu=None, std=None)

In [145]:
learn_env.data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2000 entries, 2010-01-19 to 2017-12-26
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   EUR=    2000 non-null   float64
 1   r       2000 non-null   float64
 2   s       2000 non-null   float64
 3   m       2000 non-null   float64
 4   v       2000 non-null   float64
dtypes: float64(5)
memory usage: 93.8 KB


In [146]:
valid_env = Finance(symbol, features, window=learn_env.window,
                        lags=learn_env.lags, leverage=learn_env.leverage,
                        min_performance=learn_env.min_performance,
                        start=a + b, end=a + b + c,
                        mu=learn_env.mu, std=learn_env.std)

In [147]:
valid_env.data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 500 entries, 2017-12-27 to 2019-12-20
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   EUR=    500 non-null    float64
 1   r       500 non-null    float64
 2   s       500 non-null    float64
 3   m       500 non-null    float64
 4   v       500 non-null    float64
dtypes: float64(5)
memory usage: 23.4 KB


In [148]:
set_seeds(100)
agent = FQLAgent(24, 0.0001, learn_env, valid_env)

C:\Users\hong9\anaconda3\envs\q_env\lib\site-packages\keras\optimizers\optimizer_v2\rmsprop.py:140: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [149]:
episodes = 61

In [ ]:
agent.learn(episodes)

1/1 [==============================] - 0s 65ms/step


1/1 [==============================] - 0s 32ms/step


1/1 [==============================] - 0s 63ms/step


1/1 [==============================] - 0s 48ms/step


1/1 [==============================] - 0s 49ms/step


...

1/1 [==============================] - 0s 62ms/step


1/1 [==============================] - 0s 69ms/step


1/1 [==============================] - 0s 60ms/step


1/1 [==============================] - 0s 61ms/step


1/1 [==============================] - 0s 53ms/step


In [ ]:
agent.epsilon

In [ ]:
plt.figure(figsize=(10, 6))
x = range(1, len(agent.averages) + 1)
y = np.polyval(np.polyfit(x, agent.averages, deg=3), x)
plt.plot(agent.averages, label='moving average')
plt.plot(x, y, 'r--', label='regression')
plt.xlabel('episodes')
plt.ylabel('total reward')
plt.legend();

In [ ]:
plt.figure(figsize=(10, 6))
x = range(1, len(agent.performances) + 1)
y = np.polyval(np.polyfit(x, agent.performances, deg=3), x)
y_ = np.polyval(np.polyfit(x, agent.vperformances, deg=3), x)
plt.plot(agent.performances[:], label='training')
plt.plot(agent.vperformances[:], label='validation')
plt.plot(x, y, 'r--', label='regression (train)')
plt.plot(x, y_, 'r-.', label='regression (valid)')
plt.xlabel('episodes')
plt.ylabel('gross performance')
plt.legend();